In [1]:
import requests
import pandas as pd
import geopandas as gpd
import requests
import zipfile
import io
import csv
import urllib, subprocess
import json
from io import BytesIO, StringIO

# Three phases: 1: bring in data. 2: Prepare data for analysis. 3: Determine number of trees to plant per census tract

### There are 6 "sets" of data that will be brought in for this project
### ACS data (7 parameters): Hispanic population, Black population, households with income below the poverty level,
### Seniors (>62 years old), young children (<5 years old), people who commute by bus, people who walk to work
### Landsat 8 to convert to land surface temperature
### PM 2.5 air quality data
### Average annual daily traffic
### Tree canopy coverage
### MKE census tracts to be used as the study extent

# Phase 1: Bring in all data sources

## process to bring in ACS Data

In [2]:
# start of process to bring in acs data. Had to make two api requests as household income below poverty level in a seperate api call
pd.options.display.max_columns = None # show all columns in display

def json_to_dataframe(response): # create function to convert from json to dataframe
    """
    Convert response to dataframe
    """
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [3]:
# creating workspace
workspace = arcpy.env.workspace = r'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb'
working_dir = r'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb'

In [4]:
# define census data source and where to fetch csv data
# census_url has all acs variables except for households in poverty
census_url = r"https://api.census.gov/data/2020/acs/acs5/profile?get=NAME,DP03_0062E,DP05_0024E,DP05_0005E,DP05_0071E,DP05_0038E,DP03_0021E,DP03_0022E&for=tract&in=state:55%20county:079"

# households in poverty url
house_url = r"https://api.census.gov/data/2021/acs/acs5?get=B05010_002E&for=tract&in=state:55%20county:079"

In [5]:
# bring in data
census_response = requests.get(census_url)
census_response.json

house_response = requests.get(house_url)
house_response.json

<bound method Response.json of <Response [200]>>

In [7]:
# this is all acs data minus median household income
mke_acs_data = json_to_dataframe(census_response)
mke_acs_data['tract'] = mke_acs_data['tract'].str.lstrip('0') # need to remove leading zeros from tracts so I can join future data to this
mke_acs_data.head(5)

,NAME,DP03_0062E,DP05_0024E,DP05_0005E,DP05_0071E,DP05_0038E,DP03_0021E,DP03_0022E,state,county,tract
0,"Census Tract 1.01, Milwaukee County, Wisconsin",30708,709,260,169,3819,173,43,55,079,101
1,"Census Tract 1.02, Milwaukee County, Wisconsin",43487,612,183,331,1902,84,7,55,079,102
2,"Census Tract 2.01, Milwaukee County, Wisconsin",33451,348,638,728,3358,92,72,55,079,201
3,"Census Tract 2.02, Milwaukee County, Wisconsin",68576,576,809,228,3820,0,0,55,079,202
4,"Census Tract 3.01, Milwaukee County, Wisconsin",69833,251,83,63,111,0,3,55,079,301


In [8]:
# bring in median household income data
house_acs_data = json_to_dataframe(house_response)
house_acs_data['tract'] = mke_acs_data['tract'].str.lstrip('0') # need to remove leading zeros from tracts so I can join future data to this
house_acs_data.head()

,B05010_002E,state,county,tract
0,714,55,079,101
1,59,55,079,102
2,1043,55,079,201
3,251,55,079,202
4,3,55,079,301


In [9]:
# join the fields so all acs is in one layer
df_merged = mke_acs_data.merge(house_acs_data, left_on='tract', right_on='tract')
df_merged.head()

,NAME,DP03_0062E,DP05_0024E,DP05_0005E,DP05_0071E,DP05_0038E,DP03_0021E,DP03_0022E,state_x,county_x,tract,B05010_002E,state_y,county_y
0,"Census Tract 1.01, Milwaukee County, Wisconsin",30708,709,260,169,3819,173,43,55,079,101,714,55,079
1,"Census Tract 1.02, Milwaukee County, Wisconsin",43487,612,183,331,1902,84,7,55,079,102,59,55,079
2,"Census Tract 2.01, Milwaukee County, Wisconsin",33451,348,638,728,3358,92,72,55,079,201,1043,55,079
3,"Census Tract 2.02, Milwaukee County, Wisconsin",68576,576,809,228,3820,0,0,55,079,202,251,55,079
4,"Census Tract 3.01, Milwaukee County, Wisconsin",69833,251,83,63,111,0,3,55,079,301,3,55,079


In [ ]:
# export as csv
df_merged.to_csv(r'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\data\acs_joined_data.csv')

## Process to bring in Census Tracts

In [15]:
# bring in data from big 10 location
mke_census_tracts_big = r"https://gis-mclio.opendata.arcgis.com/datasets/d60028f77427442cac14d8f6effab8b1_1.zip"

In [16]:
local_path = 'C:/Users/swimb/Documents/UMN/fall2022/GIS5571/mketreecover/mketrecover/mketrecover.gdb/'
print('Downloading shapefile...')
r = requests.get(mke_census_tracts_big)
z = zipfile.ZipFile(io.BytesIO(r.content))
print("Done")

z.extractall(path=local_path) # extract to folder
filenames = [y for y in sorted(z.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
print(filenames)


# https://medium.com/@loldja/reading-shapefile-zips-from-a-url-in-python-3-93ea8d727856

Done
['Census_Tracts.dbf', 'Census_Tracts.prj', 'Census_Tracts.shp', 'Census_Tracts.shx']


In [17]:
# works in juypter notebook but not in arcpy environment
dbf, prj, shp, shx = [filename for filename in filenames]
mke_census_tract = gpd.read_file(local_path + shp)
print("Shape of the dataframe: {}".format(mke_census_tract.shape))
print("Projection of dataframe: {}".format(mke_census_tract.crs))
mke_census_tract.tail(20) #last 5 records in dataframe

ImportError: the 'read_file' function requires the 'fiona' package, but it is not installed or does not import correctly.
Importing fiona resulted in: No module named 'fiona'

In [ ]:
ax = mke_census_tract.plot()
ax.set_title("MKE Census Tracts. Default view)");

## Process to bring in traffic data

In [18]:
# traffic data bringing in . Works in juypter notebook

traffic_url = r"https://dotmaps.wi.gov/arcgis/rest/services/agohub/TRAFFIC_COUNTS/MapServer/0/query?outFields=*&where=1%3D1&f=geojson"
traffic_data = gpd.read_file(traffic_url)
traffic_data.to_file('traffic_url')

traffic_data

ImportError: the 'read_file' function requires the 'fiona' package, but it is not installed or does not import correctly.
Importing fiona resulted in: No module named 'fiona'

# Process to bring in Landsat 8 imagery. I tried several different ways. 
## I found the most success with trial 3

In [ ]:
# trial 1
import ee
pip install earthengine-api
pip install earthengine-api --upgrade

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
# was not working

dataset = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filterDate('2014-07-01', '2-15-07-31');
trueColor432 = dataset.select(['B4', 'B5', 'B10']);
trueColor432Vis = {
  min: 0.0,
  max: 0.4,};
Map.addLayer()

In [ ]:
# trial 2. This got me the farthest as I selected the right scene, but when trying to download, I ran into problems that others had and could not get past
# https://github.com/yannforget/landsatxplore
pip install landsatxplore

In [ ]:
import json
from landsatxplore.api import API

# Initialize a new API instance and get an access key
api = API('bott0137@umn.edu', 'THIS IS WHERE I PUT MY PASSWORD, BUT DID NOT INCLUDE FOR PRIVACY')

# Search for Landsat TM scenes
scenes = api.search(
    dataset='landsat_8_c1',
    latitude=43.038902,
    longitude=-87.906471,
    start_date='2014-07-01',
    end_date='2014-07-28',
    max_cloud_cover=10
)

print(f"{len(scenes)} scenes found.")

# Process the result
for scene in scenes:
    print(scene['acquisition_date'].strftime('%Y-%m-%d'))
    # Write scene footprints to disk
    fname = f"{scene['landsat_product_id']}.geojson"
    with open(fname, "w") as f:
        json.dump(scene['spatial_coverage'].__geo_interface__, f)

api.logout()

# https://community.esri.com/t5/python-questions/quot-list-index-out-of-range-quot-for-downloading/td-p/1174082

In [ ]:
# this should download the scene I want
# was not working and this forum had the same issue with no one able to resolve it: https://community.esri.com/t5/python-questions/quot-list-index-out-of-range-quot-for-downloading/td-p/1174082
# getting list index out of range, this is frustrating!

from landsatxplore.earthexplorer import EarthExplorer

ee = EarthExplorer('bott0137@umn.edu', 'THIS IS WHERE I PUT MY PASSWORD, BUT DID NOT INCLUDE FOR PRIVACY')

ee.download(scene_id='LC80230302014202LGN01', output_dir='C:/Users/swimb/Documents/UMN/fall2022/GIS5571')

ee.logout()

In [ ]:
# trial 3, I got the farthest I thought from this method, but it would not download
# import Google earth engine module
import ee

#Authenticate the Google earth engine with google account
ee.Authenticate()
ee.Initialize()

In [ ]:
#load image
MyImage  =  ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

In [ ]:
#create function to download the image or imagecollection as you desire
def downloader(ee_object,region): 
    try:
        #download image
        if isinstance(ee_object, ee.image.Image):
            print('Its Image')
            url = ee_object.getDownloadUrl({
                    'scale': 30,
                    'crs': 'EPSG:4326',
                    'region': region
                })
            return url
        
        #download imagecollection
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            print('Its ImageCollection')
            ee_object_new = ee_object.mosaic()
            url = ee_object_new.getDownloadUrl({
                    'scale': 30,
                    'crs': 'EPSG:4326',
                    'region': region
                })
            return url
    except:
        print("Could not download")

In [ ]:
geometry = ee.Geometry.Rectangle([80.058, 26.347, 82.201, 28.447])

region = geometry.toGeoJSONString()#region must in JSON format

path = downloader(MyImage,region)#call function

print(path)#print the download URL

In [ ]:
# trial 4
#import module 
import ee

#intialize the google earth api
ee.Initialize()

geometry = ee.Geometry.Rectangle([80.0, 26.3, 82.2, 28.4]) 

region = geometry.toGeoJSONString()#region must in JSON format


# Get a download URL for an image.
imageCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
imageCollection  = imageCollection .mosaic()
url = imageCollection .getDownloadUrl({
                    'scale': 5,
                    'crs': 'EPSG:4326',
                    'region': region
                })
print(url)

# Phase 2: Prepare data for analysis. Mainly need to get data summarized by census tract

### Get tree canopy data into percet by each tract and tree canopy area for each tract

In [ ]:
# zonal stats to get tree cover percent for each census tract
arcpy.sa.ZonalStatisticsAsTable("Census_Tracts", "Tract_ID_I", "USA NLCD Tree Canopy Cover", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\ZonalSt_Census_1", "DATA", "MEAN", "CURRENT_SLICE", [90], "AUTO_DETECT", "ARITHMETIC", 360)

In [ ]:
# join mean percentage of each census tract to CENSUS TRACTS Layer
arcpy.management.JoinField("Census_Tracts", "Tract_ID_I", "ZonalSt_Census_1", "Tract_ID_I", "MEAN", "NOT_USE_FM", None)

In [ ]:
# calculate the area of each census tract in sq km
arcpy.management.CalculateGeometryAttributes("MKE_census", "AreaKM AREA_GEODESIC", '', "SQUARE_KILOMETERS", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "SAME_AS_INPUT")

In [ ]:
# multiple the percent tree cover by .01 and by the area of census tract to get the sq km tree coverage of each census tract
arcpy.management.CalculateField("MKE_census", "canopySqKm", "!AreaKM! * !percentcover! * 0.01", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

### A map was made showing the percentage tree canopy cover in mke by census tract

## Process air quality data for analysis

In [ ]:
# extract by mask all 18 rasters to get into MKE county extent
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_201801_201812-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201812_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_200101_200112-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200112_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_200201_200212-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200212_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_200301_200312-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A2003212_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_200401_200412-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200412_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_200501_200512-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200512_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_200601_200612-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200612_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_200701_200712-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200712_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_200801_200812-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200812_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_200901_200912-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200912_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_201001_201012-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201012_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_201101_201112-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201112_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_201201_201212-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201212_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_201301_201312-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201312_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_201401_201412-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201412_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_201501_201512-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201512_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_201601_201612-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201612_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_201701_201712-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201712_RH35")
out_raster = arcpy.sa.ExtractByMask(r"raw air quality\V4NA03_PM25_NA_200001_200012-RH35-NoNegs.asc", "County_Boundary", "INSIDE", '-88.0699458650294 42.8420241385384 -87.8237083357211 43.1925666611169 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200012_RH35")

In [ ]:
# resample each of these. As they were not perfect squares
arcpy.management.Resample(r"clipped air quality\A200012_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200012_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A200112_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201012_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A200212_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200212_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A2003212_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200312_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A200412_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200412_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A200512_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200512_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A200612_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200612_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A200712_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200712_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A200812_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200812_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A200912_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A200912_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A201012_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201012_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A201112_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201112_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A201212_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201212_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A201312_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201312_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A201412_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201412_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A201512_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201512_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A201612_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201612_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A201712_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201712_RH35_Resample", "0.001 0.001", "NEAREST")
arcpy.management.Resample(r"clipped air quality\A201812_RH35", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\A201812_RH35_Resample", "0.001 0.001", "NEAREST")

In [ ]:
# cell stats resampled. This combined all rasters into one by taking the mean of all 18 years of data
out_raster = arcpy.sa.CellStatistics(r"clippedairqualityresample\A200012_RH35_Resample;clippedairqualityresample\A200212_RH35_Resample;clippedairqualityresample\A200312_RH35_Resample;clippedairqualityresample\A200412_RH35_Resample;clippedairqualityresample\A200512_RH35_Resample;clippedairqualityresample\A200612_RH35_Resample;clippedairqualityresample\A200712_RH35_Resample;clippedairqualityresample\A200812_RH35_Resample;clippedairqualityresample\A200912_RH35_Resample;clippedairqualityresample\A201012_RH35_Resample;clippedairqualityresample\A201112_RH35_Resample;clippedairqualityresample\A201212_RH35_Resample;clippedairqualityresample\A201312_RH35_Resample;clippedairqualityresample\A201412_RH35_Resample;clippedairqualityresample\A201512_RH35_Resample;clippedairqualityresample\A201612_RH35_Resample;clippedairqualityresample\A201712_RH35_Resample;clippedairqualityresample\A201812_RH35_Resample", "MEAN", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\CellStats_resampled")

In [ ]:
# zonal stats as table of this data to get air quality data by census tract
arcpy.sa.ZonalStatisticsAsTable("MKE_census", "Tract_ID_I", "CellStats_resampled", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\ZonalStats_mke", "DATA", "MEAN", "CURRENT_SLICE", [90], "AUTO_DETECT", "ARITHMETIC", 360)

In [ ]:
# join to mke_census
arcpy.management.JoinField("MKE_census", "Tract_ID_I", "MKE_census_SummarizeWithin", "Tract_ID_I", "MEAN", "NOT_USE_FM", None)

# Process landsat 8. Convert it to Land surface temperature raster

In [ ]:
# determine NDVI
output_raster = arcpy.sa.RasterCalculator(' Float("Band_5" - "Band_4") / Float("Band_5" +"Band_4")'); output_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\NDVI_1")

In [ ]:
# calculate TOA
output_raster = arcpy.sa.RasterCalculator('0.0003342 * "Band_10"  +  0.1'); output_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\TOA_1")

In [ ]:
# calculate Brightness temperature BT
output_raster = arcpy.sa.RasterCalculator('(1321.0789 / Ln((774.8853 / "TOA_1") + 1)) - 273.15'); output_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\BT_1")

In [ ]:
# calculate proportion of vegetation
output_raster = arcpy.sa.RasterCalculator(' Square(("NDVI_1" + 0.248754) / (0.660537 + 0.248754))'); output_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\PV_1")

In [ ]:
# calcluate land surface emissivity
output_raster = arcpy.sa.RasterCalculator('0.004 * "PV_1" + 0.986'); output_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\EC_1")

In [ ]:
# output_raster = arcpy.sa.RasterCalculator(' ("BT_1" / (1 + (0.00115 * "BT_1" / 1.4388) * Ln("EC_1")))'); output_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\LST_1")

In [ ]:
# calculate land surface temperature
output_raster = arcpy.sa.RasterCalculator(' ("BT_1" / (1 + (0.00115 * "BT_1" / 1.4388) * Ln("EC_1")))'); output_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\LST_1")

In [ ]:
# convert celcius to fahrenheit
output_raster = arcpy.sa.RasterCalculator('( "LST_1" * (9/5)) + 32'); output_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\LST_1_F")

In [ ]:
# zonal stats as table for LST
arcpy.sa.ZonalStatisticsAsTable("Tree Planting Priority", "Tract_ID_I", "LST_1_F", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\LST_F_Final_1", "DATA", "MEAN", "CURRENT_SLICE", [90], "AUTO_DETECT", "ARITHMETIC", 360)

In [ ]:
# join to MKE_Census
arcpy.management.JoinField("Tree Planting Priority", "Tract_ID_I", "LST_F_Final_1", "Tract_ID_I", "MEAN", "NOT_USE_FM", None)

# Process traffic data

In [ ]:
# clip data to just be in milwaukee
arcpy.analysis.Clip("Traffic_Counts", "MKE_census", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\MKETrafficCounts_12_6", None)

In [ ]:
# count number of vehicles that go by roads in each census tract
arcpy.analysis.SummarizeWithin("MKE_census", "MKETrafficCounts", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\mketreecover\mketrecover\mketrecover.gdb\MKE_census_SummarizeWithin", "KEEP_ALL", "Traffic1 Sum", "NO_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

# Process ACS Data

In [ ]:
# join acs data to mke census layer
arcpy.management.JoinField("MKE_census", "Tract_ID_I", "acs_joined_data.csv", "tract", "B05010_002E;DP03_0021E;DP03_0022E;DP03_0062E;DP05_0005E;DP05_0024E;DP05_0038E;DP05_0071E", "NOT_USE_FM", None)

In [ ]:
# create scatter plot showing relationship between MEDIAN HOUSEHOLD INCOME and Average percent tree canopy coverage
# make another map showing relationship between avg percent tree canopy coverage and senior population

In [ ]:
# COMPUTE PERCENTAGES FOR demographics variables. divide the variable sum of the census tract by the same varibale sum for the whole county. Manually read sum from acs data new field of SUM
arcpy.management.CalculateField("MKE_census", "pHispanic", "(!HispanicPop! / 146352) * 100", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "pNonHspBlack", "(!BlackPop! / 246274) * 100", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "pSeniors", "(!F62YearsOlder! / 160413) * 100", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "pYoungChildren", "(!Under5Years! / 65114) * 100", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "pWalk2Work", "(!Walked! / 14870) * 100", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "pBus2Work", "(!PublicTransportation! / 21079) * 100", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "pTreeCanopy", "(!canopySqKm! / 96.85643883542919) * 100", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "pPoverty", "(!TotalUnderPoverty! / 32716) * 100", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
# made map showing relationship betwen percent poverty and percent tree canopy cover

In [ ]:
# compute disparity indices. subtract variable by pTree Canopy
arcpy.management.CalculateField("MKE_census", "DisIdxPoverty", "(!pPoverty! - !pTreeCanopy!)", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "DisIdxHispanic", "(!pHispanic! - !pTreeCanopy!)", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "DisIdxNonHspBlack", "(!pNonHspBlack! - !pTreeCanopy!)", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "DisIdxSeniors", "(!pSeniors! - !pTreeCanopy!)", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "DisIdxYoungChildren", "(!pYoungChildren! - !pTreeCanopy!)", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "DisIdxWalk2Work", "(!pWalk2Work! - !pTreeCanopy!)", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("MKE_census", "DisIdxBus2Work", "(!pBus2Work! - !pTreeCanopy!)", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
# creat map showing relationship between african american and tree canopy cover by standard deviation

# Third and final step: Suitability analysis


In [ ]:
# make suitability analysis layer
arcpy.ba.MakeSuitabilityAnalysisLayer("MKE_census_SummarizeWithin", "Tree Planting Priority1")

In [ ]:
# add field based suitability criteria
arcpy.ba.AddFieldBasedSuitabilityCriteria("Tree Planting Priority", "AvePM25;AvgAnnDailyTraffic;AvgMaxTemp;DisIdxBus2Work;DisIdxHispanic;DisIdxNonHspBlack;DisIdxPoverty;DisIdxSeniors;DisIdxWalk2Work;DisIdxYoungChildren;percentcover")


## Apply weights in suitablity analysis using business analyst
## Make map of this

### allocate trees based on priority scores and the goal of 75,000 trees planted for 2023

In [ ]:
# visually find the sum of the final score values using "statistics" in arcpro

In [ ]:
# calculate the final score/ the sum of all the final scores times the number of trees to be planted (75000)
arcpy.management.CalculateField("Tree Planting Priority", "TreesToPlant", "(!FinalScore! / 215.9) * 75000", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")

In [ ]:
# join to mke census layer
arcpy.management.JoinField("MKE_census_SummarizeWithin", "Tract_ID_I", "Tree Planting Priority", "Tract_ID_I", "TreesToPlant", "NOT_USE_FM", None)
# make final map